## Databases 

In [1]:
import os
import pandas as pd
import time

In [2]:
hostname = "127.0.0.1"

In [3]:
os.listdir("../data/testData/")

['hospital-sort-full.json', 'hospital-paths-full.json', 'college.json']

In [4]:
hospital = pd.read_json("../data/testData/hospital-sort-full.json").to_dict("records") 
hospitalAllPaths = pd.read_json("../data/testData/hospital-paths-full.json").to_dict("list")

In [5]:
college = pd.read_json("../data/testData/college.json").to_dict("records")

### Mongo 

In [6]:
from pymongo import MongoClient

In [7]:
client = MongoClient(host=hostname,port=27017)

In [8]:
client

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)

In [9]:
local = client["local"]

In [10]:
allPaths = [dict(name=k, locations=v) for k,v in hospitalAllPaths.items()]

In [11]:
local["hospital"].insert_many(hospital)

In [12]:
local["hospitalPaths"].insert_many(allPaths)

In [13]:
collegeMongo = local["college"]

In [14]:
collegeMongo.insert_many(college)

### Neo4j 

In [15]:
from py2neo import Graph, Node, Relationship

In [16]:
txpool = []
for e in hospital:
    start = Node("Hospital",name=e["start_name"],lat=e["start_lat"],lon=e["start_lon"])
    end = Node("Hospital",name=e["end_name"],lat=e["end_lat"],lon=e["end_lon"])
    
    r = Relationship(start,"NEAREST_PATH",end)
    #c = Relationship(start,"IS_CONNECTED",end,distance=e["distance"])
    txpool.append(r)
    #txpool.append(c)
    
    paths = hospitalAllPaths.get(e["start_name"])
    
    if not paths is None:
        for i in paths: 
            destination = Node("Hospital", name=i["end_name"], lat=e["end_lat"], lon=e["end_lon"]) 
            is_connected = Relationship(start, "IS_CONNECTED", destination, distance=i["distance"])
            txpool.append(is_connected)
    else: 
        continue 

In [17]:
unique = set([e["name"] for e in college])

for idx, collegeName in enumerate(unique):
    # Filter 
    pred = list(filter(lambda e: e["name"] == collegeName, college)) 
    getProgramName = list(map(lambda e: Node("ProgramName",name=e["programName"]), pred))
                              
    university = Node("University", name=pred[0]["name"], web=pred[0]["web"], city=pred[0]["city"])    
    
    [txpool.append(Relationship(university,"HAS_CAPACITY",e)) for e in getProgramName]

In [18]:
import time
graph = Graph(f"bolt://{hostname}:7687", auth=("neo4j", "12345678"))
start_time = time.time() 
for idx, e in enumerate(txpool):  
    if idx % 2 == 0:
        tx = graph.begin()
        print(f"Processing {idx} out of {len(txpool)} - {tx.create(e)}") 
        graph.commit(tx)
print(f"After commit: {time.time() - start_time}")

Processing 0 out of 23415 - None
Processing 2 out of 23415 - None
Processing 4 out of 23415 - None
Processing 6 out of 23415 - None
Processing 8 out of 23415 - None
Processing 10 out of 23415 - None
Processing 12 out of 23415 - None
Processing 14 out of 23415 - None
Processing 16 out of 23415 - None
Processing 18 out of 23415 - None
Processing 20 out of 23415 - None
Processing 22 out of 23415 - None
Processing 24 out of 23415 - None
Processing 26 out of 23415 - None
Processing 28 out of 23415 - None
Processing 30 out of 23415 - None
Processing 32 out of 23415 - None
Processing 34 out of 23415 - None
Processing 36 out of 23415 - None
Processing 38 out of 23415 - None
Processing 40 out of 23415 - None
Processing 42 out of 23415 - None
Processing 44 out of 23415 - None
Processing 46 out of 23415 - None
Processing 48 out of 23415 - None
Processing 50 out of 23415 - None
Processing 52 out of 23415 - None
Processing 54 out of 23415 - None
Processing 56 out of 23415 - None
Processing 58 out o

Processing 564 out of 23415 - None
Processing 566 out of 23415 - None
Processing 568 out of 23415 - None
Processing 570 out of 23415 - None
Processing 572 out of 23415 - None
Processing 574 out of 23415 - None
Processing 576 out of 23415 - None
Processing 578 out of 23415 - None
Processing 580 out of 23415 - None
Processing 582 out of 23415 - None
Processing 584 out of 23415 - None
Processing 586 out of 23415 - None
Processing 588 out of 23415 - None
Processing 590 out of 23415 - None
Processing 592 out of 23415 - None
Processing 594 out of 23415 - None
Processing 596 out of 23415 - None
Processing 598 out of 23415 - None
Processing 600 out of 23415 - None
Processing 602 out of 23415 - None
Processing 604 out of 23415 - None
Processing 606 out of 23415 - None
Processing 608 out of 23415 - None
Processing 610 out of 23415 - None
Processing 612 out of 23415 - None
Processing 614 out of 23415 - None
Processing 616 out of 23415 - None
Processing 618 out of 23415 - None
Processing 620 out o

Processing 1102 out of 23415 - None
Processing 1104 out of 23415 - None
Processing 1106 out of 23415 - None
Processing 1108 out of 23415 - None
Processing 1110 out of 23415 - None
Processing 1112 out of 23415 - None
Processing 1114 out of 23415 - None
Processing 1116 out of 23415 - None
Processing 1118 out of 23415 - None
Processing 1120 out of 23415 - None
Processing 1122 out of 23415 - None
Processing 1124 out of 23415 - None
Processing 1126 out of 23415 - None
Processing 1128 out of 23415 - None
Processing 1130 out of 23415 - None
Processing 1132 out of 23415 - None
Processing 1134 out of 23415 - None
Processing 1136 out of 23415 - None
Processing 1138 out of 23415 - None
Processing 1140 out of 23415 - None
Processing 1142 out of 23415 - None
Processing 1144 out of 23415 - None
Processing 1146 out of 23415 - None
Processing 1148 out of 23415 - None
Processing 1150 out of 23415 - None
Processing 1152 out of 23415 - None
Processing 1154 out of 23415 - None
Processing 1156 out of 23415

Processing 1644 out of 23415 - None
Processing 1646 out of 23415 - None
Processing 1648 out of 23415 - None
Processing 1650 out of 23415 - None
Processing 1652 out of 23415 - None
Processing 1654 out of 23415 - None
Processing 1656 out of 23415 - None
Processing 1658 out of 23415 - None
Processing 1660 out of 23415 - None
Processing 1662 out of 23415 - None
Processing 1664 out of 23415 - None
Processing 1666 out of 23415 - None
Processing 1668 out of 23415 - None
Processing 1670 out of 23415 - None
Processing 1672 out of 23415 - None
Processing 1674 out of 23415 - None
Processing 1676 out of 23415 - None
Processing 1678 out of 23415 - None
Processing 1680 out of 23415 - None
Processing 1682 out of 23415 - None
Processing 1684 out of 23415 - None
Processing 1686 out of 23415 - None
Processing 1688 out of 23415 - None
Processing 1690 out of 23415 - None
Processing 1692 out of 23415 - None
Processing 1694 out of 23415 - None
Processing 1696 out of 23415 - None
Processing 1698 out of 23415

Processing 2196 out of 23415 - None
Processing 2198 out of 23415 - None
Processing 2200 out of 23415 - None
Processing 2202 out of 23415 - None
Processing 2204 out of 23415 - None
Processing 2206 out of 23415 - None
Processing 2208 out of 23415 - None
Processing 2210 out of 23415 - None
Processing 2212 out of 23415 - None
Processing 2214 out of 23415 - None
Processing 2216 out of 23415 - None
Processing 2218 out of 23415 - None
Processing 2220 out of 23415 - None
Processing 2222 out of 23415 - None
Processing 2224 out of 23415 - None
Processing 2226 out of 23415 - None
Processing 2228 out of 23415 - None
Processing 2230 out of 23415 - None
Processing 2232 out of 23415 - None
Processing 2234 out of 23415 - None
Processing 2236 out of 23415 - None
Processing 2238 out of 23415 - None
Processing 2240 out of 23415 - None
Processing 2242 out of 23415 - None
Processing 2244 out of 23415 - None
Processing 2246 out of 23415 - None
Processing 2248 out of 23415 - None
Processing 2250 out of 23415

...

Processing 20860 out of 23415 - None
Processing 20862 out of 23415 - None
Processing 20864 out of 23415 - None
Processing 20866 out of 23415 - None
Processing 20868 out of 23415 - None
Processing 20870 out of 23415 - None
Processing 20872 out of 23415 - None
Processing 20874 out of 23415 - None
Processing 20876 out of 23415 - None
Processing 20878 out of 23415 - None
Processing 20880 out of 23415 - None
Processing 20882 out of 23415 - None
Processing 20884 out of 23415 - None
Processing 20886 out of 23415 - None
Processing 20888 out of 23415 - None
Processing 20890 out of 23415 - None
Processing 20892 out of 23415 - None
Processing 20894 out of 23415 - None
Processing 20896 out of 23415 - None
Processing 20898 out of 23415 - None
Processing 20900 out of 23415 - None
Processing 20902 out of 23415 - None
Processing 20904 out of 23415 - None
Processing 20906 out of 23415 - None
Processing 20908 out of 23415 - None
Processing 20910 out of 23415 - None
Processing 20912 out of 23415 - None
P

Processing 21438 out of 23415 - None
Processing 21440 out of 23415 - None
Processing 21442 out of 23415 - None
Processing 21444 out of 23415 - None
Processing 21446 out of 23415 - None
Processing 21448 out of 23415 - None
Processing 21450 out of 23415 - None
Processing 21452 out of 23415 - None
Processing 21454 out of 23415 - None
Processing 21456 out of 23415 - None
Processing 21458 out of 23415 - None
Processing 21460 out of 23415 - None
Processing 21462 out of 23415 - None
Processing 21464 out of 23415 - None
Processing 21466 out of 23415 - None
Processing 21468 out of 23415 - None
Processing 21470 out of 23415 - None
Processing 21472 out of 23415 - None
Processing 21474 out of 23415 - None
Processing 21476 out of 23415 - None
Processing 21478 out of 23415 - None
Processing 21480 out of 23415 - None
Processing 21482 out of 23415 - None
Processing 21484 out of 23415 - None
Processing 21486 out of 23415 - None
Processing 21488 out of 23415 - None
Processing 21490 out of 23415 - None
P

Processing 21998 out of 23415 - None
Processing 22000 out of 23415 - None
Processing 22002 out of 23415 - None
Processing 22004 out of 23415 - None
Processing 22006 out of 23415 - None
Processing 22008 out of 23415 - None
Processing 22010 out of 23415 - None
Processing 22012 out of 23415 - None
Processing 22014 out of 23415 - None
Processing 22016 out of 23415 - None
Processing 22018 out of 23415 - None
Processing 22020 out of 23415 - None
Processing 22022 out of 23415 - None
Processing 22024 out of 23415 - None
Processing 22026 out of 23415 - None
Processing 22028 out of 23415 - None
Processing 22030 out of 23415 - None
Processing 22032 out of 23415 - None
Processing 22034 out of 23415 - None
Processing 22036 out of 23415 - None
Processing 22038 out of 23415 - None
Processing 22040 out of 23415 - None
Processing 22042 out of 23415 - None
Processing 22044 out of 23415 - None
Processing 22046 out of 23415 - None
Processing 22048 out of 23415 - None
Processing 22050 out of 23415 - None
P

Processing 22566 out of 23415 - None
Processing 22568 out of 23415 - None
Processing 22570 out of 23415 - None
Processing 22572 out of 23415 - None
Processing 22574 out of 23415 - None
Processing 22576 out of 23415 - None
Processing 22578 out of 23415 - None
Processing 22580 out of 23415 - None
Processing 22582 out of 23415 - None
Processing 22584 out of 23415 - None
Processing 22586 out of 23415 - None
Processing 22588 out of 23415 - None
Processing 22590 out of 23415 - None
Processing 22592 out of 23415 - None
Processing 22594 out of 23415 - None
Processing 22596 out of 23415 - None
Processing 22598 out of 23415 - None
Processing 22600 out of 23415 - None
Processing 22602 out of 23415 - None
Processing 22604 out of 23415 - None
Processing 22606 out of 23415 - None
Processing 22608 out of 23415 - None
Processing 22610 out of 23415 - None
Processing 22612 out of 23415 - None
Processing 22614 out of 23415 - None
Processing 22616 out of 23415 - None
Processing 22618 out of 23415 - None
P

Processing 23112 out of 23415 - None
Processing 23114 out of 23415 - None
Processing 23116 out of 23415 - None
Processing 23118 out of 23415 - None
Processing 23120 out of 23415 - None
Processing 23122 out of 23415 - None
Processing 23124 out of 23415 - None
Processing 23126 out of 23415 - None
Processing 23128 out of 23415 - None
Processing 23130 out of 23415 - None
Processing 23132 out of 23415 - None
Processing 23134 out of 23415 - None
Processing 23136 out of 23415 - None
Processing 23138 out of 23415 - None
Processing 23140 out of 23415 - None
Processing 23142 out of 23415 - None
Processing 23144 out of 23415 - None
Processing 23146 out of 23415 - None
Processing 23148 out of 23415 - None
Processing 23150 out of 23415 - None
Processing 23152 out of 23415 - None
Processing 23154 out of 23415 - None
Processing 23156 out of 23415 - None
Processing 23158 out of 23415 - None
Processing 23160 out of 23415 - None
Processing 23162 out of 23415 - None
Processing 23164 out of 23415 - None
P

In [19]:
graph.run("MATCH (n) RETURN COUNT(n)")

COUNT(n)
14256


In [20]:
#graph.run("MATCH (n) DETACH DELETE(n)")